In [2]:
import argparse
import os
import random
import shutil
import time
import warnings
import pickle
import numpy as np
import math
import sys
import copy
import re
import pandas as pd
import matplotlib.pyplot as plt
import json
import cv2
from itertools import compress

import torch
import torch.nn as nn
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor,DefaultTrainer,HookBase
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer,ColorMode,GenericMask
from detectron2.structures import BoxMode
from detectron2.evaluation import COCOEvaluator,inference_on_dataset
from detectron2.data import build_detection_test_loader,DatasetMapper,build_detection_train_loader,MetadataCatalog,DatasetCatalog
import detectron2.data.transforms as T
import detectron2.utils.comm as comm

import ray
import time

import uuid as uuid
from operator import itemgetter
import seaborn as sns

import shapely
import shapely.geometry
from shapely.geometry import Polygon,MultiPolygon,GeometryCollection
from shapely.validation import make_valid
from shapely.geometry import mapping
#import geopandas as gpd

#import imgfileutils as imf
#import segmentation_tools as sgt
from aicsimageio import AICSImage, imread
from skimage import measure, segmentation
from skimage.measure import regionprops
from skimage.color import label2rgb
#import progressbar
from IPython.display import display, HTML
#from MightyMosaic import MightyMosaic

import glob
from PIL import Image
import csv

In [3]:
# setup directory
root = r'/Users/lovely_shufan/'

project_dir = root + r'Dropbox (Edison_Lab@UGA)/AMF/AMF Imaging 2022/0_inference_using_MaskRCNN_2021/'
output_dir = project_dir + r'2_infer_result/GA_GWAS_2022/'

model_dir = root + r'Dropbox (Edison_Lab@UGA)/AMF/AMF Imaging 2021/2_computer_vision/'

data_dir = r'/Volumes/easystore/GWAS 2022/'

blocks = ['Block2/','Block3/','Block8/']

## Model Inference Configuration

In [4]:
classes=['root','AMF internal hypha','AMF external hypha','AMF arbuscule','AMF vesicle','AMF spore','others']

In [5]:
cfg = get_cfg() # return default configuration
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")) # copy config files from open source projects

# training configuration
cfg.DATASETS.TEST=()
cfg.DATALOADER.NUM_WORKERS=2
#cfg.SOLVER.IMS_PER_BATCH=args.batch_size

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE=128 #Number of regions per image used to train RPN. faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES=len(classes)# (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
cfg.MODEL.BACKBONE.FREEZE_AT=2
cfg.SEED=1
cfg.AUG_FLAG=1

# inference configuration
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8  # set threshold for this model
cfg.MODEL.WEIGHTS=os.path.join(model_dir, "Trainset1_model_best.pth") # path to the best model trained
cfg.MODEL.DEVICE='cpu' # use cpu for inference

# I removed spore and others from inference classes
inf_metadata = MetadataCatalog.get("inference").set(thing_classes=['root','AMF internal hypha','AMF external hypha','AMF arbuscule','AMF vesicle'])


In [6]:
predictor = DefaultPredictor(cfg)

[10/01 23:10:14 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /Users/lovely_shufan/Dropbox (Edison_Lab@UGA)/AMF/AMF Imaging 2021/2_computer_vision/Trainset1_model_best.pth ...


The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


## Ray remote funtion

In [85]:
a = [1,2,3]
diff_array = np.array(a).reshape(3, 1, 1)
print(diff_array)
expanded_array = np.broadcast_to(diff_array, (3,1920,2560))
#print(expanded_array)
print(expanded_array.shape)
diff = expanded_array.transpose(1, 2, 0)
#print(diff)
print(diff.shape)

[[[1]]

 [[2]]

 [[3]]]
(3, 1920, 2560)
(1920, 2560, 3)


In [81]:
testczi = AICSImage(allpath2block3img[126])

In [84]:
print(testczi.dims)

<Dimensions [T: 1, C: 1, Z: 1, Y: 6909, X: 11284, S: 3]>


In [82]:
testimg = testczi.get_image_data("YXS", T=0,C=0,Z=0)

In [98]:
print(testimg.shape)
print(type(testimg))

(6909, 11284, 3)
<class 'numpy.ndarray'>


In [87]:
def centering2train(diff, img, x, y):
    '''
    :param diff: 3d tensor (row, col, 3)
    :param img: np.ndarray (row, col ,3)
    :return img:
    :rtype ndnumpy.array:
    Objective: output an image centered using training set per-channel means
    '''
    
    # Convert diff to a numpy array and reshape it to (3, 1, 1)
    diff_array = np.array(diff).reshape(3, 1, 1)
    # Broadcast to shape (3, imageHeight, imgWidth)
    expanded_array = np.broadcast_to(diff_array, (3, y, x))
    # Transpose to get shape (imageHeight, imgWidth, 3)
    diff_ts = expanded_array.transpose(1, 2, 0)
    
    img = np.add(img,diff_ts)
    # Clip values outside the interval are clipped to the interval edges
    np.clip(img, 0, 255, out=img)
    return img

def padImg(img, x, y, tilex, tiley):
    '''
    :param img:
    :return padded img:
    :rtype ndnumpy.array:
    Objective: output a padded image dividle by tile size
    '''
    pad_top = tiley - (y % tiley)
    pad_lft = tilex - (x % tilex)
    img = cv2.copyMakeBorder(img,pad_top,0,pad_lft,0,cv2.BORDER_CONSTANT,value=[0,0,0])
    return img

In [97]:
@ray.remote
def inference(pathtofile, block, diff, predictor):
    blklist = []
    imgidlist = []
    sceneidlist = []
    tileidlist = []
    classlist = []
    confscorelist=[]
    arealist = []

    # read in czi
    czi = AICSImage(pathtofile)
   
    for scene in czi.scenes:
        # extract image by scene
        czi.set_scene(scene)
        img = czi.get_image_data("YXS", T=0,C=0,Z=0) # numpy.ndarray  
        y = img.shape[0]
        x = img.shape[1]
        # centering
        img = centering2train(diff, img, x, y)
        # pad image
        img = padImg(img, x, y, 2560, 1920)
        # tiling
        for i in range(0,y,1920):
            for j in range(0,x,2560):
                xmin = j
                xmax = j + 2560
                ymin = i
                ymax = i + 1920
                tile_id = str(xmin)+"_"+str(ymin)+"_"+str(xmax)+"_"+str(ymax)
                subimg = img[ymin:ymax,xmin:xmax]
                outputs = predictor(subimg)
                        
                #inference outputs
                clasind = outputs['instances'].get('pred_classes')
                allmasks = outputs['instances'].get('pred_masks')
                allscores = outputs['instances'].get('scores')
            
                num_seg = clasind.size()[0]
                if num_seg != 0: # only save an entry when the image contains a segmentation
                    blklist = blklist + np.repeat(block[:-1], num_seg).tolist()
                    imgidlist = imgidlist + np.repeat(pathtofile, num_seg).tolist()
                    sceneidlist = sceneidlist + np.repeat(scene, num_seg).tolist()
                    tileidlist = tileidlist + np.repeat(tile_id, num_seg).tolist()
                    confscorelist = confscorelist + allscores.tolist()
                
                    # calculate the area of segmentation
                    v = Visualizer(subimg[:, :, ::-1], MetadataCatalog.get("inference"), scale=1.0)
                    for i in range(0,num_seg,1):
                        #calculate mask area
                        locmask = np.asarray(allmasks[i,:,:])
                        gmask = GenericMask(locmask,v.output.height,v.output.width)
                        if gmask.polygons:
                            mergpolygon = gmask.polygons[0]
                            all_points_x = mergpolygon[::2]
                            all_points_y = mergpolygon[1::2]
                            pgon = Polygon(zip(all_points_x,all_points_y))
                            arealist.append(pgon.area)
                            # class index to class name
                            classlist.append(classes[clasind.tolist()[i]])
                        else: # assign NAs to non-polygon mask
                            arealist.append(math.nan)
                            classlist.append(math.nan)

    # export inference result as df
    infresults = pd.DataFrame({
    'block': blklist,
    'filename': imgidlist,
    'scene': sceneidlist,
    'tile': tileidlist,
    'annotations': classlist,
    'area': arealist,
    'confidenceScore': confscorelist})
    
    # delete 
    return infresults

## Prepare for parallel inference

In [9]:
allpath2block2img = [os.path.join(path,name) for path, dirs, files in os.walk(os.path.join(data_dir,'Block2/'))
                        for name in files
                        if name.endswith('.czi')]
allpath2block3img = [os.path.join(path,name) for path, dirs, files in os.walk(os.path.join(data_dir,'Block3/'))
                        for name in files
                        if name.endswith('.czi')]
allpath2block8img = [os.path.join(path,name) for path, dirs, files in os.walk(os.path.join(data_dir,'Block8/'))
                        for name in files
                        if name.endswith('.czi')]

In [10]:
print(len(allpath2block2img))
print(len(allpath2block3img))
print(len(allpath2block8img))

344
333
384


In [89]:
# centering values calculated in script 4
diffs = [[10.12746633, 18.06137262, 27.510427], 
         [16.24330208, 23.29101365, 34.06829238], 
         [10.1628206,  16.92133011, 26.18143223]]

## Trial run on Block 3, two scenes per image

In [43]:
ray.init(num_cpus=18, object_store_memory=int(2e9), ignore_reinit_error=True)

2023-10-02 00:05:12,036	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.8.18
Ray version:,2.6.3
Dashboard:,http://127.0.0.1:8265


In [44]:
results_blk3 = list()

In [34]:
ids_blk3 = [inference.remote(path, 'Block3/', diffs[1], predictor) 
        for path in allpath2block3img]

#results_blk3 = list() #creates a new dataframe that's empty
t0 = time.time()
for i in range(1,400,1):
    ready, not_ready = ray.wait(ids_blk3, num_returns = 1)
    print('image:', i) 
    results_blk3.append(ray.get(ready))
    del ready #%% Clean-up object store data 
    ids_blk3 = not_ready
    if not ids_blk3: 
        continue
    print('Time Elapsed:\t{:.4f}'.format(time.time() - t0))
print('Total Time Elapsed:\t{:.4f}'.format(time.time() - t0))

(raylet) Spilled 2186 MiB, 13 objects, write throughput 875 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.
(raylet) Spilled 4205 MiB, 25 objects, write throughput 969 MiB/s.
(raylet) Spilled 8746 MiB, 52 objects, write throughput 1037 MiB/s.
(raylet) Spilled 16651 MiB, 99 objects, write throughput 823 MiB/s.
(raylet) Spilled 32967 MiB, 196 objects, write throughput 715 MiB/s.


image: 1
Time Elapsed:	115.4381
image: 2
Time Elapsed:	117.6237
image: 3
Time Elapsed:	126.5018
image: 4
Time Elapsed:	137.3074
image: 5
Time Elapsed:	184.2561
image: 6
Time Elapsed:	226.0420
image: 7
Time Elapsed:	271.3528
image: 8
Time Elapsed:	317.1990
image: 9
Time Elapsed:	368.1811
image: 10
Time Elapsed:	375.3132
image: 11
Time Elapsed:	379.5779
image: 12
Time Elapsed:	406.7882
image: 13
Time Elapsed:	445.7507
image: 14
Time Elapsed:	455.7605
image: 15
Time Elapsed:	529.3461
image: 16
Time Elapsed:	568.1768
image: 17
Time Elapsed:	584.7673
image: 18
Time Elapsed:	588.7704
image: 19
Time Elapsed:	601.3358
image: 20
Time Elapsed:	639.8641
image: 21
Time Elapsed:	654.8863
image: 22
Time Elapsed:	724.5197
image: 23
Time Elapsed:	783.4055
image: 24
Time Elapsed:	790.4936
image: 25
Time Elapsed:	861.4857
image: 26
Time Elapsed:	875.0980
image: 27
Time Elapsed:	875.7832
image: 28
Time Elapsed:	889.2013


KeyboardInterrupt: 

In [35]:
ray.shutdown()

## batch the number of tasks to prevent memory leaks
(detectron2-py38) lovely_shufan@Shufans-iMac-6 ~ % ray memory

Plasma memory usage 1850 MiB, 11 objects, 97.0% full, 26.46% needed

Spilled 40031 MiB, 238 objects, avg write throughput 697 MiB/s

Restored 672 MiB, 4 objects, avg read throughput 1111 MiB/s

Objects consumed by Ray tasks: 1850 MiB.

Object fetches queued, waiting for available memory.


Based on the output of ray memory command, I need to reduce the number of concurrently running tasks to 10 at a time. 


In [58]:
ray.init(num_cpus=18, object_store_memory=int(2e9), ignore_reinit_error=True)

2023-10-02 00:18:17,117	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.8.18
Ray version:,2.6.3
Dashboard:,http://127.0.0.1:8265


In [59]:
results_blk3 = list()

batch_size = 15
t0 = time.time()
for i in range(0, len(allpath2block3img), batch_size):
    batch = allpath2block3img[i:i+batch_size] # if the remaining images are less than 10, the last batch will be smaller
    ids = [inference.remote(path, 'Block3/', diffs[1], predictor) 
        for path in batch]
    
    t1 = time.time()
    for task in range(1,20,1):
        ready, not_ready = ray.wait(ids, num_returns = 1)
        ntask = i + task
        print('image:', ntask) 
        results_blk3.append(ray.get(ready))
        del ready #%% Clean-up object store data 
        ids = not_ready
        if not ids: 
            break
        print('Time Elapsed:\t{:.4f}'.format(time.time() - t0))
    del ids
    print('Batch Time Elapsed:\t{:.4f}'.format(time.time() - t0))
print('Total Time Elapsed:\t{:.4f}'.format(time.time() - t0))

image: 1
Time Elapsed:	1785.8471
image: 2
Time Elapsed:	2037.0238
image: 3
Time Elapsed:	2046.7596
image: 4
Time Elapsed:	2392.2758
image: 5
Time Elapsed:	2876.9302
image: 6
Time Elapsed:	3084.7077
image: 7
Time Elapsed:	3256.4126
image: 8
Time Elapsed:	4058.3812
image: 9
Time Elapsed:	4914.0737
image: 10
Time Elapsed:	4986.8173
image: 11
Time Elapsed:	5130.5874
image: 12
Time Elapsed:	5783.0540
image: 13
Time Elapsed:	5967.2372
image: 14
Time Elapsed:	6181.9847
image: 15
Batch Time Elapsed:	7796.6167


(raylet) Spilled 2186 MiB, 13 objects, write throughput 734 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.


image: 16
Time Elapsed:	10186.0594
image: 17
Time Elapsed:	10557.4724
image: 18
Time Elapsed:	10558.7226
image: 19
Time Elapsed:	10600.7853
image: 20
Time Elapsed:	10766.6246
image: 21
Time Elapsed:	10885.6464
image: 22
Time Elapsed:	11014.7239
image: 23
Time Elapsed:	13329.8404
image: 24
Time Elapsed:	13351.0565
image: 25
Time Elapsed:	13377.4381
image: 26
Time Elapsed:	13591.1445
image: 27
Time Elapsed:	13637.6264
image: 28
Time Elapsed:	13692.7116
image: 29
Time Elapsed:	13787.9270
image: 30
Batch Time Elapsed:	15304.1210
image: 31
Time Elapsed:	17685.6775
image: 32
Time Elapsed:	17868.3662
image: 33
Time Elapsed:	17986.3179
image: 34
Time Elapsed:	18103.6428
image: 35
Time Elapsed:	18149.8605
image: 36
Time Elapsed:	18515.3288
image: 37
Time Elapsed:	18635.6918
image: 38
Time Elapsed:	20493.3800
image: 39
Time Elapsed:	21036.7094
image: 40
Time Elapsed:	21081.6163
image: 41
Time Elapsed:	21160.5849
image: 42
Time Elapsed:	21188.1041
image: 43
Time Elapsed:	21534.6503
image: 44
Time

(raylet) Spilled 4373 MiB, 26 objects, write throughput 907 MiB/s.


image: 46
Time Elapsed:	24544.8937
image: 47
Time Elapsed:	25042.6702
image: 48
Time Elapsed:	25374.5600
image: 49
Time Elapsed:	25433.0516
image: 50
Time Elapsed:	25448.9829
image: 51
Time Elapsed:	25806.0140
image: 52
Time Elapsed:	26510.9858
image: 53
Time Elapsed:	27203.0450
image: 54
Time Elapsed:	27384.8791
image: 55
Time Elapsed:	27821.5933
image: 56
Time Elapsed:	27923.3316
image: 57
Time Elapsed:	28305.7249
image: 58
Time Elapsed:	28401.0903
image: 59
Time Elapsed:	28523.0148
image: 60
Batch Time Elapsed:	28581.9654
image: 61
Time Elapsed:	31059.9927
image: 62
Time Elapsed:	31130.3565
image: 63
Time Elapsed:	31354.5801
image: 64
Time Elapsed:	31747.8290
image: 65
Time Elapsed:	31761.9848
image: 66
Time Elapsed:	31980.8465
image: 67
Time Elapsed:	32834.8445
image: 68
Time Elapsed:	33767.8651
image: 69
Time Elapsed:	33844.2957
image: 70
Time Elapsed:	34283.1315
image: 71
Time Elapsed:	34420.4836
image: 72
Time Elapsed:	34482.1630
image: 73
Time Elapsed:	34682.4373
image: 74
Time

(raylet) Spilled 8410 MiB, 50 objects, write throughput 1049 MiB/s.


image: 91
Time Elapsed:	46017.0691
image: 92
Time Elapsed:	46438.6736
image: 93
Time Elapsed:	46645.8124
image: 94
Time Elapsed:	46669.4195
image: 95
Time Elapsed:	46781.5148
image: 96
Time Elapsed:	47024.5580
image: 97
Time Elapsed:	47120.3720
image: 98
Time Elapsed:	48826.9132
image: 99
Time Elapsed:	48872.1145
image: 100
Time Elapsed:	49234.5433
image: 101
Time Elapsed:	49592.7153
image: 102
Time Elapsed:	49652.1050
image: 103
Time Elapsed:	49675.4643
image: 104
Time Elapsed:	49896.4967
image: 105
Batch Time Elapsed:	50861.4131
image: 106
Time Elapsed:	53437.3085
image: 107
Time Elapsed:	53675.0254
image: 108
Time Elapsed:	53813.3434
image: 109
Time Elapsed:	53842.5960
image: 110
Time Elapsed:	53873.7610
image: 111
Time Elapsed:	53923.6936
image: 112
Time Elapsed:	53953.0787
image: 113
Time Elapsed:	55861.1141
image: 114
Time Elapsed:	56153.4473
image: 115
Time Elapsed:	56446.9078
image: 116
Time Elapsed:	56447.6855
image: 117
Time Elapsed:	56515.9282
image: 118
Time Elapsed:	56910.

RayTaskError(ValueError): [36mray::inference()[39m (pid=33153, ip=127.0.0.1)
  File "/var/folders/mv/p4jr8xs52gl7rk_wf4pn6mn80000gn/T/ipykernel_32189/69097058.py", line 64, in inference
  File "/Users/lovely_shufan/opt/anaconda3/envs/detectron2-py38/lib/python3.8/site-packages/pandas/core/frame.py", line 709, in __init__
    mgr = dict_to_mgr(data, index, columns, dtype=dtype, copy=copy, typ=manager)
  File "/Users/lovely_shufan/opt/anaconda3/envs/detectron2-py38/lib/python3.8/site-packages/pandas/core/internals/construction.py", line 481, in dict_to_mgr
    return arrays_to_mgr(arrays, columns, index, dtype=dtype, typ=typ, consolidate=copy)
  File "/Users/lovely_shufan/opt/anaconda3/envs/detectron2-py38/lib/python3.8/site-packages/pandas/core/internals/construction.py", line 115, in arrays_to_mgr
    index = _extract_index(arrays)
  File "/Users/lovely_shufan/opt/anaconda3/envs/detectron2-py38/lib/python3.8/site-packages/pandas/core/internals/construction.py", line 655, in _extract_

In [68]:
ray.shutdown()

## Output block 3 inference result

In [69]:
print(len(results_blk3))

126


In [61]:
df_blk3 = results_blk3[0][0]
for i in range(1,len(results_blk3),1):
    newdf = results_blk3[i][0]
    df_blk3 = pd.concat([df_blk3, newdf], axis=0)

In [64]:
print(len(df_blk3['filename'].unique()))

126


In [65]:
df_blk3.to_csv(os.path.join(output_dir,"block3_segmentation_image_1to126.txt"),index=False)

## Debug:  ValueError: All arrays must be of the same length
Inference failed on image 127 from block3, because, in inference line 64, column lengths are not the same. 

In [94]:
def local_inference(pathtofile, block, diff, predictor):
    blklist = []
    imgidlist = []
    sceneidlist = []
    tileidlist = []
    classlist = []
    confscorelist=[]
    arealist = []

    # read in czi
    czi = AICSImage(pathtofile)
   
    for scene in czi.scenes:
        
        print(scene)
        
        # extract image by scene
        czi.set_scene(scene)
        img = czi.get_image_data("YXS", T=0,C=0,Z=0) # numpy.ndarray      
        y = img.shape[0]
        x= img.shape[1]
        # centering
        img = centering2train(diff, img, x, y)
        # pad image
        img = padImg(img, x, y, 2560, 1920)
        # tiling
        for i in range(0,y,1920):
            for j in range(0,x,2560):
                xmin = j
                xmax = j + 2560
                ymin = i
                ymax = i + 1920
                tile_id = str(xmin)+"_"+str(ymin)+"_"+str(xmax)+"_"+str(ymax)
                
                print(tile_id)
                
                subimg = img[ymin:ymax,xmin:xmax]
                outputs = predictor(subimg)
                        
                #inference outputs
                clasind = outputs['instances'].get('pred_classes')
                allmasks = outputs['instances'].get('pred_masks')
                allscores = outputs['instances'].get('scores')
                
                num_seg = clasind.size()[0]
                if num_seg != 0: # only save an entry when the image contains a segmentation
                    blklist = blklist + np.repeat(block[:-1], num_seg).tolist()
                    imgidlist = imgidlist + np.repeat(pathtofile, num_seg).tolist()
                    sceneidlist = sceneidlist + np.repeat(scene, num_seg).tolist()
                    tileidlist = tileidlist + np.repeat(tile_id, num_seg).tolist()
                    confscorelist = confscorelist + allscores.tolist()
                
                    # calculate the area of segmentation
                    v = Visualizer(subimg[:, :, ::-1], MetadataCatalog.get("inference"), scale=1.0)
                    for i in range(0,num_seg,1):
                        #calculate mask area
                        locmask = np.asarray(allmasks[i,:,:])
                        gmask = GenericMask(locmask,v.output.height,v.output.width)
                        if gmask.polygons:
                            mergpolygon = gmask.polygons[0]
                            all_points_x = mergpolygon[::2]
                            all_points_y = mergpolygon[1::2]
                            pgon = Polygon(zip(all_points_x,all_points_y))
                            arealist.append(pgon.area)
                            # class index to class name
                            classlist.append(classes[clasind.tolist()[i]])
                        else:
                            arealist.append(math.nan)
                            classlist.append(math.nan)
                            
        print("blkList length = ", len(blklist))
        print("imgidList length = ", len(imgidlist))
        print("sceneidList length = ", len(sceneidlist))
        print("tileidList length = ", len(tileidlist))
        print("classList length = ", len(classlist))
        print("areaList length = ", len(arealist))
        print("confscoreList length = ", len(confscorelist))
    # export inference result as df
    infresults = pd.DataFrame({
    'block': blklist,
    'filename': imgidlist,
    'scene': sceneidlist,
    'tile': tileidlist,
    'annotations': classlist,
    'area': arealist,
    'confidenceScore': confscorelist})
    
    # delete 
    return infresults

In [95]:
results_testimg = local_inference(allpath2block3img[126],'Block3/', diffs[1], predictor)


ScanRegion0
0_0_2560_1920
2560_0_5120_1920
5120_0_7680_1920
7680_0_10240_1920
10240_0_12800_1920
0_1920_2560_3840
2560_0_5120_1920
5120_0_7680_1920
7680_0_10240_1920
10240_0_12800_1920
0_3840_2560_5760
2560_0_5120_1920
5120_0_7680_1920
7680_0_10240_1920
10240_0_12800_1920
0_5760_2560_7680
2560_5760_5120_7680
5120_0_7680_1920
7680_0_10240_1920
10240_0_12800_1920
blkList length =  3
imgidList length =  3
sceneidList length =  3
tileidList length =  3
classList length =  3
areaList length =  3
confscoreList length =  3
ScanRegion1
0_0_2560_1920
2560_0_5120_1920
5120_0_7680_1920
7680_0_10240_1920
10240_0_12800_1920
0_1920_2560_3840
2560_0_5120_1920
5120_0_7680_1920
7680_0_10240_1920
10240_0_12800_1920
0_3840_2560_5760
2560_3840_5120_5760
5120_3_7680_1923
7680_3_10240_1923
10240_3_12800_1923
0_5760_2560_7680
2560_5760_5120_7680
5120_0_7680_1920
7680_0_10240_1920
10240_0_12800_1920
blkList length =  9
imgidList length =  9
sceneidList length =  9
tileidList length =  9
classList length =  9


0_0_2560_1920
2560_0_5120_1920
5120_0_7680_1920
0_1920_2560_3840
2560_3_5120_1923
5120_3_7680_1923
0_3840_2560_5760
2560_0_5120_1920
5120_0_7680_1920
blkList length =  110
imgidList length =  110
sceneidList length =  110
tileidList length =  110
classList length =  110
areaList length =  110
confscoreList length =  110
ScanRegion17
0_0_2560_1920
2560_0_5120_1920
5120_0_7680_1920
7680_0_10240_1920
10240_0_12800_1920
0_1920_2560_3840
2560_0_5120_1920
5120_0_7680_1920
7680_0_10240_1920
10240_0_12800_1920
0_3840_2560_5760
2560_3840_5120_5760
5120_4_7680_1924
7680_4_10240_1924
10240_4_12800_1924
0_5760_2560_7680
2560_5760_5120_7680
5120_1_7680_1921
7680_1_10240_1921
10240_1_12800_1921
0_7680_2560_9600
2560_7680_5120_9600
5120_7680_7680_9600
7680_7680_10240_9600
10240_0_12800_1920
blkList length =  119
imgidList length =  119
sceneidList length =  119
tileidList length =  119
classList length =  119
areaList length =  119
confscoreList length =  119
ScanRegion18
0_0_2560_1920
2560_0_5120_19

In [ ]:
ray.init(num_cpus=18, object_store_memory=int(2e9), ignore_reinit_error=True)

In [ ]:
results_blk8 = list()

batch_size = 15
t0 = time.time()
for i in range(0, len(allpath2block8img), batch_size):
    batch = allpath2block8img[i:i+batch_size] # if the remaining images are less than 10, the last batch will be smaller
    ids = [inference.remote(path, 'Block8/', diffs[2], predictor) 
        for path in batch]
    
    t1 = time.time()
    for task in range(1,20,1):
        ready, not_ready = ray.wait(ids, num_returns = 1)
        ntask = i + task
        print('image:', ntask) 
        results_blk8.append(ray.get(ready))
        del ready #%% Clean-up object store data 
        ids = not_ready
        if not ids: 
            break
        print('Time Elapsed:\t{:.4f}'.format(time.time() - t0))
    print('Batch Time Elapsed:\t{:.4f}'.format(time.time() - t0))
print('Total Time Elapsed:\t{:.4f}'.format(time.time() - t0))

In [ ]:
ray.shutdown()